In [1]:
import gct,os
import numpy as np

In [2]:
names = sorted([u for u in gct.list_local_graph() if u.startswith("LFR_ud_wu_N")], key=lambda u: int(u.split("_")[3][1:]))
len(names)

32

In [3]:
names[:2]

['LFR_ud_wu_N32', 'LFR_ud_wu_N38']

In [4]:
names[:3]

['LFR_ud_wu_N32', 'LFR_ud_wu_N38', 'LFR_ud_wu_N45']

In [5]:
def run_alg(algname):
    for dataname in names:
        print("running", algname,dataname)
        retcode = os.system("timeout -k 1h 1h python run_alg.py {} {} 0 >  /data/log/{}_{}.log 2>&1".format(algname,dataname, algname,dataname))
        if retcode !=0:
            break


In [6]:
import numpy as np
algs =gct.list_algorithms()
np.random.shuffle(algs)
len(algs)

79

In [ ]:
run_alg("alg_GossipMap")

In [ ]:
run_alg("alg_pg_label_propagation")

In [13]:
from joblib import Parallel, delayed
ret=Parallel(n_jobs=6)(delayed(run_alg)(x) for x in algs)    

In [11]:
algs={'cdc_DEMON',
 'cdc_HDEMON',
 'karateclub_BigClam',
 'karateclub_DANMF',
 'karateclub_EdMot',
 'karateclub_EgoNetSplitter',
 'karateclub_NNSED',
 'karateclub_SCD',
 'karateclub_SymmNMF'}
from joblib import Parallel, delayed
ret=Parallel(n_jobs=3)(delayed(run_alg)(x) for x in algs)    

In [14]:
import numpy as np
np.where(not ret), len(ret)

((array([], dtype=int64),), 72)

In [18]:
results = {}
d = gct.list_all_clustering_results(print_format=False)
for u in d:
    if "LFR_ud_wu_N" in u:
        for run in d[u]:
            a = gct.load_clustering_result(u, run)
            results[u + run] = a

len(results)

2293

In [19]:
import pandas as pd
df = pd.DataFrame([(u['runname'],u['dataname'],u['timecost']) for u in results.values()], columns=['alg','datanem','timecost'])

In [20]:
missedargs = set(gct.list_algorithms()).difference(df['alg'].values)
missedargs


set()

In [21]:
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    display (df.sort_values("timecost"))

,alg,datanem,timecost
16,igraph_community_label_propagation,LFR_ud_wu_N32,0.000108
95,igraph_community_label_propagation,LFR_ud_wu_N38,0.000142
252,igraph_community_label_propagation,LFR_ud_wu_N54,0.000165
173,igraph_community_label_propagation,LFR_ud_wu_N45,0.000185
82,igraph_community_fastgreedy,LFR_ud_wu_N38,0.000196
...,...,...,...
2234,karateclub_EdMot,LFR_ud_wu_N7093,2705.017424
1041,snap_Girvan_Newman,LFR_ud_wu_N321,2838.634086
1927,karateclub_SCD,LFR_ud_wu_N2851,3026.575022
2251,pycabem_HiReCS,LFR_ud_wu_N8511,3281.394896


In [22]:
df['N']=df['datanem'].map(lambda u: int(u.split("_")[-1][1:])*1)

In [23]:
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    pdf=df.pivot_table(columns=['N'], index=['alg'], values='timecost')
    for x in missedargs:
        pdf=pdf.append(pd.Series(name=x))

    display(pdf)

N,32,38,45,54,64,76,91,109,130,156,187,224,268,321,385,462,554,664,796,955,1146,1375,1650,1980,2376,2851,3421,4105,4926,5911,7093,8511
alg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
alg_GossipMap,0.824268,0.578304,0.568582,0.587782,0.581059,0.574404,0.582474,0.576562,0.578278,0.590472,0.570507,0.597648,0.581933,0.589078,0.623898,0.761993,0.813419,0.914003,1.151156,1.277835,1.824309,2.215926,3.577725,5.389516,8.203985,10.758113,19.595577,25.854883,37.838256,58.724374,72.203342,173.631825
alg_Paris,0.032778,0.010189,0.007304,0.009709,0.023100,0.032177,0.049471,0.059838,0.162449,0.209572,0.361169,0.557903,0.522210,1.071120,1.057277,1.634652,2.949438,3.273313,5.410474,6.133387,8.326878,11.200937,16.454067,25.396638,59.937348,76.846738,60.020527,139.715234,180.084143,318.057504,409.541208,669.819985
alg_RelaxMap,0.017512,0.016086,0.015355,0.014822,0.015438,0.015915,0.016772,0.018916,0.019466,0.022619,0.027554,0.031698,0.034198,0.047393,0.070786,0.092785,0.257092,0.300339,0.363904,0.608192,0.821721,1.128855,1.826701,2.525286,4.609402,5.037076,7.021940,11.110696,15.631536,40.465141,33.991031,51.793577
alg_lso_cluster,0.034042,0.015298,0.012834,0.021068,0.039511,0.031583,0.014866,0.058316,0.016146,0.022651,0.038273,0.022274,0.041298,0.042392,0.050166,0.069071,0.081769,0.106856,0.133398,0.207788,0.313434,0.533960,0.796495,1.169308,1.639185,2.451594,3.949411,5.551823,8.000281,12.211201,18.776037,27.651468
alg_pg_label_propagation,0.567224,0.386774,0.387496,0.379540,0.378323,0.378760,0.381339,0.391613,0.389583,0.385203,0.389332,0.388767,0.391355,0.392180,0.392246,0.404419,0.409346,0.433617,0.475689,0.538652,0.641391,0.794861,0.962903,1.250959,1.672893,2.216454,3.076671,4.334209,6.019761,8.673788,12.621398,18.182889
alg_streamcom,0.014884,NaN,0.013224,NaN,0.008747,0.013355,0.013686,0.013899,NaN,0.013737,0.009202,0.009837,0.009993,0.016768,0.018285,0.019759,0.022827,0.027066,0.032741,0.038286,NaN,0.075707,NaN,0.143244,0.183922,0.259075,0.398611,0.584610,NaN,1.262741,1.911813,2.961613
cdc_CONGA,0.458922,0.481340,0.454060,0.454291,0.428232,0.531330,0.526274,0.546207,0.618212,0.653437,0.430299,0.355906,0.355829,0.385115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cdc_CliquePercolation,0.043021,0.026423,0.014344,0.035420,0.016911,0.018176,0.038397,0.065318,0.071705,0.043819,0.050657,0.152876,0.249915,0.331741,0.318632,0.655843,0.910796,2.629626,3.963860,6.230598,10.502218,22.354250,30.173617,68.088387,107.015173,161.021673,468.817672,927.325002,1012.193228,1282.627199,2491.340366,NaN
cdc_Connected_Iterative_Scan,0.027399,0.016686,0.023724,0.027678,0.035484,0.045662,0.076621,0.131183,0.127732,0.362525,0.566528,0.818161,1.762459,1.982048,4.027131,7.132011,16.830673,28.629064,30.804997,89.262164,179.909904,271.568698,464.604487,497.854862,1301.310503,2655.433465,NaN,NaN,NaN,NaN,NaN,NaN
